In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection

# 连接到 Milvus
connections.connect("default", host="10.30.28.171", port="19530")

# 定义字段
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=512)
]

collection_name = 'text_image_embeddings'
# 创建集合
schema = CollectionSchema(fields, description="Image-Text Embeddings Collection")
collection = Collection(name=collection_name, schema=schema)

# 创建索引
index_params = {"index_type": "IVF_FLAT", "metric_type": "COSINE", "params": {"nlist": 1024}}
collection.create_index(field_name="embedding", index_params=index_params)

In [ ]:
from towhee import pipe, ops, DataCollection
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
import numpy as np

# 连接到 Milvus
connections.connect("default", host="10.30.28.171", port="19530")

# 定义 Milvus 中的集合 schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="img_embedding", dtype=DataType.FLOAT_VECTOR, dim=512),
    FieldSchema(name="text_embedding", dtype=DataType.FLOAT_VECTOR, dim=512)
]
schema = CollectionSchema(fields, "A collection for image and text embeddings")

collection_name = 'text_image_embeddings2'
# 创建 Milvus 集合
collection = Collection(collection_name, schema)


In [ ]:
from towhee import pipe, ops, DataCollection

# 定义嵌入管道
in_pipe = pipe.input('url', 'text')

# 获取图像嵌入
img_embedding = (
    in_pipe.map('url', 'img', ops.image_decode.cv2_rgb())
    .map('img', 'img_embedding', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='image'))
)

# 获取文本嵌入
text_embedding = in_pipe.map('text', 'text_embedding', ops.image_text_embedding.clip(model_name='clip_vit_base_patch16', modality='text'))

# 合并嵌入
img_text_embedding = (
    img_embedding.concat(text_embedding)
    .output('img', 'text', 'img_embedding', 'text_embedding')
)

# # 示例图像和文本
# img = 'https://towhee.io/object-detection/yolov5/raw/branch/main/test.png'
# text = 'A dog looking at a computer in bed.'
# res = img_text_embedding(img, text)
# # embeddings = DataCollection(res).to_list()
# 
# # 提取嵌入向量并存储到 Milvus
# print(res.)
# img_emb = res['img_embedding']
# text_emb = res['text_embedding']
# 
# # 插入数据
# collection.insert(collection_name, img_emb)
# collection.insert(collection_name, text_emb)

In [ ]:
# 示例图像和文本
img = 'https://towhee.io/object-detection/yolov5/raw/branch/main/test.png'
text = 'A dog looking at a computer in bed.'
res = img_text_embedding(img, text)
embeddings = DataCollection(res).to_list()

# 从结果中提取嵌入向量
img_emb = embeddings[0]['img_embedding']
text_emb = embeddings[0]['text_embedding']

# # 插入数据到 Milvus
# entities = [
#     {"name": "img_embedding", "values": [img_emb], "type": DataType.FLOAT_VECTOR},
#     {"name": "text_embedding", "values": [text_emb], "type": DataType.FLOAT_VECTOR}
# ]
# collection.insert(entities)

# 插入数据到 Milvus
entities = [
    [img_emb],
    [text_emb]
]
collection.insert(entities)


In [ ]:
# def search_similar_embeddings(query_embedding, top_k=5):
#     search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}
#     results = collection.search([query_embedding], "embedding", search_params, limit=top_k)
#     return results
# 
# 
# # 获取查询嵌入向量
# query_text = 'A dog looking at a computer in bed.'
# query_res = text_embedding(query_text)
# query_embeddings = DataCollection(query_res).to_list()
# query_emb = query_embeddings[0]['text_embedding']
# 
# # 查询最相似的 top5
# top_k_results = search_similar_embeddings(query_emb)
# for result in top_k_results[0]:
#     print(f"ID: {result.id}, Distance: {result.distance}")


In [ ]:
# 准备查询管道
def get_embedding(query, modality='image'):
    if modality == 'image':
        res = img_text_embedding(query, '')
    else:
        res = img_text_embedding('https://towhee.io/object-detection/yolov5/raw/branch/main/test.png', query)
    embedding = DataCollection(res).to_list()[0][f'{modality}_embedding']
    return embedding


# # 查询相似图片
# query_img = 'https://example.com/query_image.png'
# query_embedding = get_embedding(query_img, modality='image')
# 
# # 在 Milvus 中搜索相似图片
search_params = {"metric_type": "IP", "params": {"nprobe": 10}}
# collection.load()
# results = collection.search(
#     data=[query_embedding],
#     anns_field="img_embedding",
#     param=search_params,
#     limit=5,
#     expr=None
# )
# 
# # 打印结果
# for result in results[0]:
#     print(f"Image ID: {result.id}, Similarity: {result.distance}")

# 查询相似文本
query_text = 'A cat sitting on a chair.'
query_embedding = get_embedding(query_text, modality='text')

collection.load()
# 在 Milvus 中搜索相似文本
results = collection.search(
    data=[query_embedding],
    anns_field="text_embedding",
    param=search_params,
    limit=5,
    expr=None
)

# 打印结果
for result in results[0]:
    print(f"Text ID: {result.id}, Similarity: {result.distance}")
